In [115]:
from lxml import objectify
import pandas as pd
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy

In [116]:
# 温湿度数据处理
xml = objectify.parse('./humidity.xml')
root = xml.getroot()

data=[]
for i in range(len(root.getchildren())):
    x = root.getchildren()[i].getchildren()
    #print(x)
    data.append(x)
data = np.reshape(data,[-1,4])
#print(data.shape)
df = pd.DataFrame(data)
df.columns = ['id', 'type', 'time','value']
df['id'] = df.id.astype(int)
df['type'] = df.type.astype(str)
df["time"] = df.time.astype(str)
df['value'] = df.value.astype(str)
print(df.head())

   id      type                        time  value
0   1  02#主控室湿度  2019-07-17 16:20:54:378000  41.8%
1   2  02#主控室湿度  2019-07-17 16:09:38:398000  42.5%
2   3  02#主控室湿度  2019-07-17 15:59:32:524000  43.1%
3   4  02#主控室湿度  2019-07-17 15:49:21:611000  43.7%
4   5  02#主控室湿度  2019-07-17 15:39:10:714000  44.4%


In [117]:
data = df.copy()

In [118]:
data['value'] = data['value'].apply(lambda x: x[:-1]) # 去掉温度的单位
data['value'] = data['value'].astype(float)

data['time'] = data['time'].apply(lambda x: x[:-10]) # 精确到分

data.drop(columns='id', inplace=True)

In [120]:
print(data.head())
print('data shape:', data.shape)
print('data types:', data['type'].unique())

       type              time  value
0  02#主控室湿度  2019-07-17 16:20   41.8
1  02#主控室湿度  2019-07-17 16:09   42.5
2  02#主控室湿度  2019-07-17 15:59   43.1
3  02#主控室湿度  2019-07-17 15:49   43.7
4  02#主控室湿度  2019-07-17 15:39   44.4
data shape: (4990, 3)
data types: ['02#主控室湿度' '03#主控室湿度' '04#高压室湿度' '05#高压室湿度' '06#27.5KV室湿度' '07#10KV室湿度']


In [121]:
def abstract(data, column):
    abs_data = data.loc[data['type']==column]
    abs_data[column] = abs_data['value']
    abs_data.drop(columns=['type', 'value'], inplace=True)
    abs_data.set_index('time', inplace=True)
    return abs_data

In [122]:
#data_02 = data.loc[data['type']=='02#主控室湿度']
data_02 = abstract(data, '02#主控室湿度')
data_03 = abstract(data, '03#主控室湿度')
data_04 = abstract(data, '04#高压室湿度')
data_05 = abstract(data, '05#高压室湿度')
data_06 = abstract(data, '06#27.5KV室湿度')
data_07 = abstract(data, '07#10KV室湿度')

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/root/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [123]:
print(data_02.head())
print(data_05.head())
print(data_06.head())

                  02#主控室湿度
time                      
2019-07-17 16:20      41.8
2019-07-17 16:09      42.5
2019-07-17 15:59      43.1
2019-07-17 15:49      43.7
2019-07-17 15:39      44.4
                  05#高压室湿度
time                      
2019-07-17 16:21      50.9
2019-07-17 16:10      51.6
2019-07-17 15:59      52.0
2019-07-17 15:48      52.1
2019-07-17 15:38      52.2
                  06#27.5KV室湿度
time                          
2019-07-17 16:21          48.3
2019-07-17 16:09          49.3
2019-07-17 15:58          49.5
2019-07-17 15:48          49.9
2019-07-17 15:38          49.9


In [124]:
# 显示每个type的数据长度
for i in data['type'].unique():
    print(i + ' length: ' + str((data['type']==i).sum()))

02#主控室湿度 length: 946
03#主控室湿度 length: 949
04#高压室湿度 length: 812
05#高压室湿度 length: 683
06#27.5KV室湿度 length: 701
07#10KV室湿度 length: 899


In [128]:
re_data_02 = data_02.reset_index()
re_data_03 = data_03.reset_index()
re_data_04 = data_04.reset_index()
re_data_05 = data_05.reset_index()
re_data_06 = data_06.reset_index()
re_data_07 = data_07.reset_index()

In [129]:
con = pd.concat([re_data_02, re_data_03, re_data_04, re_data_05, re_data_06, re_data_07], axis=1)
con.head()

,time,02#主控室湿度,time,03#主控室湿度,time,04#高压室湿度,time,05#高压室湿度,time,06#27.5KV室湿度,time,07#10KV室湿度
0,2019-07-17 16:20,41.8,2019-07-17 16:20,45.1,2019-07-17 16:20,54.9,2019-07-17 16:21,50.9,2019-07-17 16:21,48.3,2019-07-17 16:21,53.9
1,2019-07-17 16:09,42.5,2019-07-17 16:10,45.2,2019-07-17 16:10,55.6,2019-07-17 16:10,51.6,2019-07-17 16:09,49.3,2019-07-17 16:11,55.2
2,2019-07-17 15:59,43.1,2019-07-17 16:00,45.6,2019-07-17 15:59,55.9,2019-07-17 15:59,52.0,2019-07-17 15:58,49.5,2019-07-17 16:01,55.7
3,2019-07-17 15:49,43.7,2019-07-17 15:49,46.3,2019-07-17 15:49,55.9,2019-07-17 15:48,52.1,2019-07-17 15:48,49.9,2019-07-17 15:51,55.4
4,2019-07-17 15:39,44.4,2019-07-17 15:39,47.0,2019-07-17 15:39,56.7,2019-07-17 15:38,52.2,2019-07-17 15:38,49.9,2019-07-17 15:40,55.9


In [130]:
con.to_csv('concat_data.csv', index=True)